In [182]:
import pandas as pd
import numpy as np

In [183]:
train = pd.read_csv('option_train.csv')
test = pd.read_csv('option_test_wolabel.csv')

In [184]:
train = train.replace({'BS': {'Under':0, 'Over':1}})

In [185]:
train.head()

,Value,S,K,tau,r,BS
0,21.670404,431.623898,420,0.341270,0.03013,0
1,0.125000,427.015526,465,0.166667,0.03126,1
2,20.691244,427.762336,415,0.265873,0.03116,0
3,1.035002,451.711658,460,0.063492,0.02972,1
4,39.553020,446.718974,410,0.166667,0.02962,0


In [186]:
train.shape

(1680, 6)

In [187]:
train.describe()

,Value,S,K,tau,r,BS
count,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000
mean,15.064384,440.901494,438.247024,0.201930,0.030235,0.436905
std,14.035795,7.527497,23.396936,0.099712,0.000557,0.496151
min,0.125000,425.472331,375.000000,0.003968,0.029510,0.000000
25%,2.255001,433.863864,420.000000,0.119048,0.029820,0.000000
50%,11.190967,442.579723,440.000000,0.202381,0.030130,0.000000
75%,25.744375,447.320414,455.000000,0.285714,0.030540,1.000000
max,60.149367,455.880619,500.000000,0.392857,0.031880,1.000000


# Creating Variables

In [192]:
new = train.copy()
new = new.drop(['BS'], axis=1)

In [193]:
col = list(new.columns)
col_r = col[::-1]

In [194]:
for i in new.columns:
    for j in col_r:
        if i==j:
            col_r.remove(i)
        else:
            new[i+'_times_'+j] = new[i]*new[j]
            new[i+'_by_'+j] = new[i]/new[j]

In [195]:
new = new.replace([np.inf, -np.inf], np.nan)

In [196]:
new.head()

,Value,S,K,tau,r,Value_times_r,Value_by_r,Value_times_tau,Value_by_tau,Value_times_K,...,S_times_tau,S_by_tau,S_times_K,S_by_K,K_times_r,K_by_r,K_times_tau,K_by_tau,tau_times_r,tau_by_r
0,21.670404,431.623898,420,0.341270,0.03013,0.652929,719.230152,7.395455,63.499325,9101.569877,...,147.300219,1264.758401,181282.037286,1.027676,12.6546,13939.595088,143.333333,1230.697675,0.010282,11.326580
1,0.125000,427.015526,465,0.166667,0.03126,0.003908,3.998720,0.020833,0.750000,58.125001,...,71.169254,2562.093150,198562.219543,0.918313,14.5359,14875.239923,77.500000,2789.999994,0.005210,5.331627
2,20.691244,427.762336,415,0.265873,0.03116,0.644739,664.032230,5.501244,77.823784,8586.866380,...,113.730463,1608.897145,177521.369606,1.030753,12.9314,13318.356868,110.337302,1560.895522,0.008285,8.532510
3,1.035002,451.711658,460,0.063492,0.02972,0.030760,34.825101,0.065714,16.301282,476.100926,...,28.680105,7114.458665,207787.362588,0.981982,13.6712,15477.792732,29.206349,7245.000056,0.001887,2.136341
4,39.553020,446.718974,410,0.166667,0.02962,1.171560,1335.348412,6.592170,237.318119,16216.738180,...,74.453163,2680.313841,183154.779504,1.089558,12.1442,13841.998650,68.333333,2459.999995,0.004937,5.626829


# Model

## Cross Validation

In [249]:
#need to work on this

## Feature Selection

In [ ]:
#need to work on this

## Modeling

In [205]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [197]:
train_final = new.copy()

In [203]:
X = train_final.values
y = train['BS'].values

In [207]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state = 1)

In [208]:
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

# Logistic Regression with all variables

In [218]:
from sklearn.linear_model import LogisticRegression
mlr = LogisticRegression(penalty='none', max_iter=1700).fit(X_train, y_train)

In [221]:
print(f'Classification Error: {1-mlr.score(X_test, y_test)}')

Classification Error: 0.05654761904761907


In [226]:
#not sure why im getting the warning for the normalized x
from sklearn.linear_model import LogisticRegression
mlr = LogisticRegression(penalty='none', max_iter=2000).fit(X_train_norm, y_train)

/Users/anushkakapur/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [223]:
print(f'Classification Error: {1-mlr.score(X_test, y_test)}')

Classification Error: 0.40476190476190477


# LDA

In [228]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [231]:
print(f'Classification Error: {1-lda.score(X_test, y_test)}')

Classification Error: 0.05654761904761907


In [233]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
lda.fit(X_train_norm, y_train)

LinearDiscriminantAnalysis()

In [235]:
print(f'Classification Error: {1-lda.score(X_test_norm, y_test)}')

Classification Error: 0.05654761904761907


# KNN

### 5 neighbors

In [241]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_norm, y_train)

KNeighborsClassifier()

In [242]:
print(f'Classification Error: {1-knn.score(X_test_norm, y_test)}')

Classification Error: 0.07738095238095233


In [247]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [248]:
print(f'Classification Error: {1-knn.score(X_test, y_test)}')

Classification Error: 0.0803571428571429


### 10 neighbors

In [245]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_norm, y_train)

KNeighborsClassifier(n_neighbors=10)

In [246]:
print(f'Classification Error: {1-knn.score(X_test_norm, y_test)}')

Classification Error: 0.0714285714285714
